In [11]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [19]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0011.mei')
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')


Successfully imported https://crimproject.org/mei/CRIM_Model_0011.mei


In [20]:
# Durations of Notes in Ngrams.  
# "2" is the minimum number to find passages of three HR chords.
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
ng = piece.getNgrams(df=dur, n=2)
ng

,Superius,Contratenor,Tenor,Bassus
0.0,"(8.0, 4.0)","(16.0, 8.0)","(72.0, 8.0)","(88.0, 8.0)"
8.0,"(4.0, 6.0)",NaN,NaN,NaN
12.0,"(6.0, 1.0)",NaN,NaN,NaN
16.0,NaN,"(8.0, 4.0)",NaN,NaN
18.0,"(1.0, 1.0)",NaN,NaN,NaN
...,...,...,...,...
865.0,"(1.0, 4.0)",NaN,"(1.0, 1.0)",NaN
866.0,"(4.0, 2.0)","(2.0, 4.0)","(1.0, 1.0)",NaN
867.0,NaN,NaN,"(1.0, 4.0)",NaN
868.0,NaN,"(4.0, 16.0)","(4.0, 16.0)","(4.0, 16.0)"


In [21]:


dur_ngrams = []

for index, rows in ng.iterrows():

    dur_ngrams_no_nan = [x for x in rows if pd.isnull(x) == False]
    dur_ngrams.append(dur_ngrams_no_nan)

ng['dur_ngrams'] = dur_ngrams
# ng['rest_count'] = rests

ng['active_voices'] = ng['dur_ngrams'].apply(len)
ng['number_dur_ngrams'] = ng['dur_ngrams'].apply(set).apply(len)
ng = ng[(ng['number_dur_ngrams'] <2) & (ng['active_voices'] > 2)]
# piece.detailIndex(ng, offset=True)
ng.head()

,Superius,Contratenor,Tenor,Bassus,dur_ngrams,active_voices,number_dur_ngrams
366.0,"(2.0, 4.0)","(2.0, 4.0)","(2.0, 4.0)",NaN,"[(2.0, 4.0), (2.0, 4.0), (2.0, 4.0)]",3,1
464.0,"(8.0, 4.0)","(8.0, 4.0)","(8.0, 4.0)",NaN,"[(8.0, 4.0), (8.0, 4.0), (8.0, 4.0)]",3,1
480.0,"(6.0, 2.0)","(6.0, 2.0)","(6.0, 2.0)",NaN,"[(6.0, 2.0), (6.0, 2.0), (6.0, 2.0)]",3,1
486.0,"(2.0, 4.0)","(2.0, 4.0)","(2.0, 4.0)",NaN,"[(2.0, 4.0), (2.0, 4.0), (2.0, 4.0)]",3,1
492.0,"(2.0, 2.0)","(2.0, 2.0)","(2.0, 2.0)",NaN,"[(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]",3,1


In [22]:
# Checks for Rests in All Voices

pd.set_option('display.max_rows', None)
nr.ffill(inplace=True)
index_of_rests = []
rests = []
for index, rows in nr.iterrows():
    rest_test = [y for y in rows if y == "Rest"]
    rests.append(rest_test)
#     index_of_rests.append(index)
nr["rests"] = rests  
nr["rests_count"] = nr["rests"].apply(len)
full_stop = nr[(nr['rests_count'] > 1) ]
rests_with_mb = piece.detailIndex(full_stop)
full_stop.head()

,Superius,Contratenor,Tenor,Bassus,rests,rests_count
0.0,G4,Rest,Rest,Rest,"[Rest, Rest, Rest]",3
8.0,G4,Rest,Rest,Rest,"[Rest, Rest, Rest]",3
12.0,B-4,Rest,Rest,Rest,"[Rest, Rest, Rest]",3
16.0,B-4,D4,Rest,Rest,"[Rest, Rest]",2
18.0,A4,D4,Rest,Rest,"[Rest, Rest]",2


In [23]:
# here we get the syllables sung at any offset
lyrics = piece.getLyric()
cols = lyrics.columns
for col in cols:
    lyrics[col] = lyrics[col].str.lower()

syll_set = []
for index2, rows2 in lyrics.iterrows():
    syll_no_nan = [z for z in rows2 if pd.isnull(z) == False]
    syll_set.append(syll_no_nan)
#     print(syll_no_nan)
lyrics['syllable_set'] = syll_set
lyrics

,Superius,Contratenor,Tenor,Bassus,syllable_set
0.0,to-,NaN,NaN,NaN,[to-]
8.0,-ta,NaN,NaN,NaN,[-ta]
12.0,pul-,NaN,NaN,NaN,[pul-]
16.0,NaN,to-,NaN,NaN,[to-]
18.0,NaN,NaN,NaN,NaN,[]
19.0,NaN,NaN,NaN,NaN,[]
20.0,NaN,NaN,NaN,NaN,[]
24.0,-chra,-ta,NaN,NaN,"[-chra, -ta]"
28.0,"es,-",pul-,NaN,NaN,"[es,-, pul-]"
32.0,NaN,NaN,NaN,NaN,[]


In [24]:
# count of voices with syllables at this offset
lyrics['active_syll_voices'] = lyrics['syllable_set'].apply(len)
# count how _many_ syllables at this offset
lyrics['number_sylls'] = lyrics['syllable_set'].apply(set).apply(len)
# get count of possible hr passages (several voices with same syllable)
lyrics_hr = lyrics[(lyrics['active_syll_voices'] > 2) & (lyrics['number_sylls'] < 2)]
# piece.detailIndex(lyrics_hr, offset=True)
# lyrics['is_hr'] = np.where(lyrics['active_voices'] > 3) 
hr_sylls_mask = lyrics_hr["active_syll_voices"]
lyrics_hr
# hr_sylls_mask

,Superius,Contratenor,Tenor,Bassus,syllable_set,active_syll_voices,number_sylls
352.0,-ris.,-ris.,-ris.,NaN,"[-ris., -ris., -ris.]",3,1
360.0,vul-,vul-,vul-,NaN,"[vul-, vul-, vul-]",3,1
366.0,-ne-,-ne-,-ne-,NaN,"[-ne-, -ne-, -ne-]",3,1
368.0,-ra-,-ra-,-ra-,NaN,"[-ra-, -ra-, -ra-]",3,1
372.0,-sti,-sti,-sti,NaN,"[-sti, -sti, -sti]",3,1
376.0,cor-,cor-,cor-,NaN,"[cor-, cor-, cor-]",3,1
464.0,in-,in-,in-,NaN,"[in-, in-, in-]",3,1
480.0,o-,o-,o-,NaN,"[o-, o-, o-]",3,1
486.0,-cu-,-cu-,-cu-,NaN,"[-cu-, -cu-, -cu-]",3,1
492.0,-rum,-rum,-rum,NaN,"[-rum, -rum, -rum]",3,1


In [25]:
# here we merge the syllable mask (where X voices have the same syllable)
# into the DF of the matching durations in 3+ voices
hr = pd.merge(ng, hr_sylls_mask, left_index=True, right_index=True)
piece.detailIndex(hr, offset=True)

Superius Contratenor       Tenor      Bassus  \
Measure Beat Offset                                                   
46      4.0  366.0   (2.0, 4.0)  (2.0, 4.0)  (2.0, 4.0)         NaN   
59      1.0  464.0   (8.0, 4.0)  (8.0, 4.0)  (8.0, 4.0)         NaN   
61      1.0  480.0   (6.0, 2.0)  (6.0, 2.0)  (6.0, 2.0)         NaN   
        4.0  486.0   (2.0, 4.0)  (2.0, 4.0)  (2.0, 4.0)         NaN   
62      3.0  492.0   (2.0, 2.0)  (2.0, 2.0)  (2.0, 2.0)         NaN   
63      3.0  500.0   (4.0, 2.0)         NaN  (4.0, 2.0)  (4.0, 2.0)   
64      2.0  506.0   (4.0, 2.0)  (4.0, 2.0)  (4.0, 2.0)  (4.0, 2.0)   
        4.0  510.0   (2.0, 4.0)  (2.0, 4.0)  (2.0, 4.0)  (2.0, 4.0)   
65      1.0  512.0   (4.0, 4.0)  (4.0, 4.0)  (4.0, 4.0)  (4.0, 4.0)   
80      1.0  632.0   (8.0, 8.0)  (8.0, 8.0)  (8.0, 8.0)  (8.0, 8.0)   
92      4.0  734.0   (2.0, 2.0)  (2.0, 2.0)         NaN  (2.0, 2.0)   
93      1.0  736.0   (2.0, 2.0)  (2.0, 2.0)  (2.0, 2.0)  (2.0, 2.0)   

                                                           dur_ngrams  \
Measure Beat Offset                                                     
46      4.0  366.0               [(2.0, 4.0), (2.0, 4.0), (2.0, 4.0)]   
59      1.0  464.0               [(8.0, 4.0), (8.0, 4.0), (8.0, 4.0)]   
61      1.0  480.0               [(6.0, 2.0), (6.0, 2.0), (6.0, 2.0)]   
        4.0  486.0               [(2.0, 4.0), (2.0, 4.0), (2.0, 4.0)]   
62      3.0  492.0               [(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]   
63      3.0  500.0               [(4.0, 2.0), (4.0, 2.0), (4.0, 2.0)]   
64      2.0  506.0   [(4.0, 2.0), (4.0, 2.0), (4.0, 2.0), (4.0, 2.0)]   
        4.0  510.0   [(2.0, 4.0), (2.0, 4.0), (2.0, 4.0), (2.0, 4.0)]   
65      1.0  512.0   [(4.0, 4.0), (4.0, 4.0), (4.0, 4.0), (4.0, 4.0)]   
80      1.0  632.0   [(8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0)]   
92      4.0  734.0               [(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]   
93      1.0  736.0   [(2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]   

                     active_voices  number_dur_ngrams  active_syll_voices  
Measure Beat Offset                                                        
46      4.0  366.0             3.0                1.0                 3.0  
59      1.0  464.0             3.0                1.0                 3.0  
61      1.0  480.0             3.0                1.0                 3.0  
        4.0  486.0             3.0                1.0                 3.0  
62      3.0  492.0             3.0                1.0                 3.0  
63      3.0  500.0             3.0                1.0                 3.0  
64      2.0  506.0             4.0                1.0                 4.0  
        4.0  510.0             4.0                1.0                 4.0  
65      1.0  512.0             4.0                1.0                 4.0  
80      1.0  632.0             4.0                1.0                 4.0  
92      4.0  734.0             3.0                1.0                 3.0  
93      1.0  736.0             4.0                1.0                 3.0